# Restaurants de Kolkata (Quaranteam)

In [3]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
%%capture
cd kolkata-restaurant

In [5]:
# Requires some path juggling to be imported
from  main import simulate

In [6]:
%%capture
cd ..

# Introduction

Cette notebook contient les resultats obtenus du projet des Restaurants de Kolkata. Ce sont des observations et des mesures quantitatives de l'effectivité des agents selon les différentes strategies. Pour commencer on va donner une explication générale sur le fonctionnement du projet.

Le projet s'effectue dans le framework déjà utilisé dans ce cours, qui utilise des ontologies et des répresentations JSON des scénarios. On a adapté ainsi le code d'une des exemples de ce framework pour simuler notre scénario.

Avant du rapport, on doit un remarques sur le problème et l'implementation qu'on a considéré. On ne permet pas les agents de changer de restaurant objectif avant d'y arriver, ce qui n'a pas été specifié. Si on permet ce comportement, alors une stratégie du type occupation minimale et distance minimale semble quasi-optimale (car on peut toujours distribuer les agents de manière quasi-parfaite et ainsi achéver un score maximal).

## 1. Agents et restaurants: répresentation et fonctionnement

On a choisi de répresenter les agents par des classes ayant les attributs suivants:

- **id:** Indice de l'agent, qui l'identifique de manière univoque.
- **player:** Réference au joueur dans le framework.
- **pos:** Position actuelle de l'agent.
- **dims:** Dimensions du scénario.
- **dir_vecs:** Vecteurs directionnels, indicant les directions valides de déplacement (ceci permettrait de modifier le mouvement des agents de manière simple et efficace).
- **walls:** Réference à la liste des obstacles du scénario.
- **occupation:** Historique d'occupation des restaurants (c'est partie des connaissances de l'agent).
- **strategy:** Stratégie à suivre dans le scenario.
- **score:** Ponctuation de l'agent.
- **current:** Position relative dans le path vers la destination actuelle.
- **moving:** Indique si l'agent est en train de se déplacer.
- **waiting:** Indique si l'agent attend dans la queue d'un restaurant.
- **goal_idx:** Indique l'indice de la destination de l'agent.
- **verbose:** Indique si on veut des indications pour chaque action prise par les agents.


On dote aussi les agents des méthodes suivantes:

- **get_goal(self, restaurants, \*args, \*\*args):** Permet de calculer la nouvelle destination.
- **find_path(self):** Trouve le chemin vers la destination actuelle.
- **simulate(self):** Simule l'avance d'un pas de temps pour l'agent.



## 2. Stratégies

On a choisir de répresenter les stratégies comme des fonctions indépendantes ("standalone") des agents. On a commencé par définir les stratégies obligatoires, c'est-à-dire les strategies *aléatoire uniforme* et *tétue*.

**Aléatoire uniforme:** Retourne toujours un restaurant de manière complètement aléatoire.

**Tétue:** Retourne toujours le même restaurant.

Pour les autres stratégies, on a choisi d'implementer:

**Distance minimale:** Cherche un des restaurants les plus proches.

**Occupation minimale:** Cherche un des restaurants qui ont la plus baisse occupation à chaque instant ('forgetful')

**Occupation minimale moyenne:** Cherche un des restaurants qui one la plus baisse occupation en moyenne.

**Strategie4:**

# Résultats des stratégies

On laisse ici les résultats, en termes du score des joueurs, des strategies quand tous les agents (10) suivent la même strategie.

In [7]:
def df_players_single_strategy(strategy,its=10):
    cols = ['Iteration', 'Cumulative Score', 'Average Score', 'Max Score', 'Min Score']
    data = {c:[] for c in cols}
    for i in range(its):
        stats = simulate(1,[strategy])
        stats = stats['players']
        data['Iteration'].append(i+1)
        data['Cumulative Score'].append(stats['cumulative'])
        data['Average Score'].append(stats['avg'])
        data['Max Score'].append(stats['max'])
        data['Min Score'].append(stats['min'])
    return pd.DataFrame.from_dict(data)

### 1. Tétue

Pour la stratégie tétue on attend en général des résultats de score très haut en moyenne, le seul problème posé étant la possible saturation des restaurants (si tous les agents choisissent le même par exemple).

In [9]:
df_tetue = df_players_single_strategy('random_obstinate')
df_tetue

,Average Score,Cumulative Score,Iteration,Max Score,Min Score
0,43.5,435,1,88,25
1,50.7,507,2,75,24
2,35.2,352,3,73,23
3,26.5,265,4,36,17
4,34.9,349,5,47,18
5,34.7,347,6,72,19
6,41.0,410,7,72,23
7,50.1,501,8,88,17
8,36.2,362,9,96,16
9,35.7,357,10,89,12


In [10]:
df_tetue.describe()

,Average Score,Cumulative Score,Iteration,Max Score,Min Score
count,10.000000,10.000000,10.00000,10.000000,10.000000
mean,38.850000,388.500000,5.50000,73.600000,19.400000
std,7.519198,75.191977,3.02765,19.097411,4.195235
min,26.500000,265.000000,1.00000,36.000000,12.000000
25%,34.975000,349.750000,3.25000,72.000000,17.000000
50%,35.950000,359.500000,5.50000,74.000000,18.500000
75%,42.875000,428.750000,7.75000,88.000000,23.000000
max,50.700000,507.000000,10.00000,96.000000,25.000000


### 2. Aléatoire uniforme

La stratégie aléatoire uniforme semble avoir un problème de base, et ceci est le temps qui est perdu pendant les déplacements des agents, qui doivent souvent changer de restaurant. D'autre côté, les attentes sont tout à fait minimisées.

In [11]:
df_aleatoire = df_players_single_strategy('uniformly_random')
df_aleatoire

,Average Score,Cumulative Score,Iteration,Max Score,Min Score
0,4.4,44,1,8,3
1,4.3,43,2,5,3
2,5.1,51,3,7,4
3,4.4,44,4,6,3
4,4.5,45,5,6,3
5,4.2,42,6,6,3
6,4.5,45,7,7,3
7,5.6,56,8,9,3
8,4.7,47,9,8,3
9,4.8,48,10,8,3


In [12]:
df_aleatoire.describe()

,Average Score,Cumulative Score,Iteration,Max Score,Min Score
count,10.000000,10.000000,10.00000,10.000000,10.000000
mean,4.650000,46.500000,5.50000,7.000000,3.100000
std,0.424918,4.249183,3.02765,1.247219,0.316228
min,4.200000,42.000000,1.00000,5.000000,3.000000
25%,4.400000,44.000000,3.25000,6.000000,3.000000
50%,4.500000,45.000000,5.50000,7.000000,3.000000
75%,4.775000,47.750000,7.75000,8.000000,3.000000
max,5.600000,56.000000,10.00000,9.000000,4.000000


### 3. Distance minimale

La stratégie de distance minimale doit se comporter de manière similaire à celle de la stratégie tétue (car une fois un restaurant a été choisi, il sera le plus proche quand la décision suivante est prise).

In [14]:
df_min_dist = df_players_single_strategy('min_distance')
df_min_dist

,Average Score,Cumulative Score,Iteration,Max Score,Min Score
0,55.7,557,1,95,29
1,37.5,375,2,99,14
2,37.6,376,3,94,17
3,55.8,558,4,97,22
4,38.5,385,5,92,21
5,46.5,465,6,98,28
6,45.6,456,7,92,22
7,38.5,385,8,94,17
8,37.9,379,9,98,28
9,47.4,474,10,100,17


In [15]:
df_min_dist.describe()

,Average Score,Cumulative Score,Iteration,Max Score,Min Score
count,10.000000,10.000000,10.00000,10.000000,10.000000
mean,44.100000,441.000000,5.50000,95.900000,21.500000
std,7.282551,72.825514,3.02765,2.884826,5.359312
min,37.500000,375.000000,1.00000,92.000000,14.000000
25%,38.050000,380.500000,3.25000,94.000000,17.000000
50%,42.050000,420.500000,5.50000,96.000000,21.500000
75%,47.175000,471.750000,7.75000,98.000000,26.500000
max,55.800000,558.000000,10.00000,100.000000,29.000000


### 4. Ocupation minimale

Pour ce stratégie, on attend des résultats similaires à la stratégie aléatoire uniforme. La raison est que, quand la pool de restaurants est suffisamment grand, le choix est pratiquement fait de la même manière. La seule différence est que le retour au même restaurant est presque toujours penalisé (car il a été sûrement occupé par l'agent au moment précedent) ce qui entraine probablement des résultats plus bas.

In [16]:
df_min_occupation = df_players_single_strategy('min_occupation_forgetful')
df_min_occupation

,Average Score,Cumulative Score,Iteration,Max Score,Min Score
0,4.9,49,1,8,3
1,4.4,44,2,6,3
2,4.4,44,3,6,3
3,4.0,40,4,6,3
4,4.2,42,5,6,3
5,4.2,42,6,10,3
6,4.5,45,7,6,3
7,5.2,52,8,8,4
8,4.1,41,9,6,3
9,5.9,59,10,10,3


In [17]:
df_min_occupation.describe()

,Average Score,Cumulative Score,Iteration,Max Score,Min Score
count,10.000000,10.000000,10.00000,10.000000,10.000000
mean,4.580000,45.800000,5.50000,7.200000,3.100000
std,0.592171,5.921711,3.02765,1.686548,0.316228
min,4.000000,40.000000,1.00000,6.000000,3.000000
25%,4.200000,42.000000,3.25000,6.000000,3.000000
50%,4.400000,44.000000,5.50000,6.000000,3.000000
75%,4.800000,48.000000,7.75000,8.000000,3.000000
max,5.900000,59.000000,10.00000,10.000000,4.000000


### 5. Occupation minimale moyenne

On attend des résultats similaires à la stratégie aleatoire uniforme encore une fois. La raison est que cette stratégie correspond à une stratégie aleatoire uniforme en priorisant les restaurants qui n'ont pas été visités encore (et ainsi en distribuant toujours les agents de manière uniforme dans les restaurants à la fin de la simulation). Le problème est ainsi le temps perdu dans des déplacements.

In [18]:
df_min_occupation_avg = df_players_single_strategy('min_occupation_avg')
df_min_occupation_avg

,Average Score,Cumulative Score,Iteration,Max Score,Min Score
0,5.2,52,1,8,4
1,4.1,41,2,5,3
2,5.4,54,3,8,4
3,4.0,40,4,6,3
4,4.6,46,5,6,3
5,4.7,47,6,6,3
6,4.4,44,7,7,3
7,4.8,48,8,9,3
8,4.3,43,9,8,3
9,5.1,51,10,6,4


In [19]:
df_min_occupation_avg.describe()

,Average Score,Cumulative Score,Iteration,Max Score,Min Score
count,10.000000,10.000000,10.00000,10.000000,10.000000
mean,4.660000,46.600000,5.50000,6.900000,3.300000
std,0.471876,4.718757,3.02765,1.286684,0.483046
min,4.000000,40.000000,1.00000,5.000000,3.000000
25%,4.325000,43.250000,3.25000,6.000000,3.000000
50%,4.650000,46.500000,5.50000,6.500000,3.000000
75%,5.025000,50.250000,7.75000,8.000000,3.750000
max,5.400000,54.000000,10.00000,9.000000,4.000000


# Interactions entre les stratégies

Ici, on peut voir les stratégies en action quand on a 2 groupes d'agents (chacun d'entre eux qui suit une stratégie différente). On va récuperer les données des agents après chaque simulation, comme on a fait pour une seule stratégie mais aussi les données des agents appartenant à chaque groupe suivant une certaine stratégie.

In [18]:
def df_interaction(n, strategies, its=10):
    assert len(strategies) == n
    cols = ['Iteration', 'Cumulative Score', 'Average Score', 'Max Score', 'Min Score']
    data = [{c:[] for c in cols} for i in range(n)]
    data.append({c:[] for c in cols})
    for i in range(its):
        stats = simulate(n, strategies)
        # Build dataframes
        for j,s in zip(range(n),strategies):
            data[j]['Iteration'].append(i+1)
            data[j]['Cumulative Score'].append(stats[s]['cumulative'])
            data[j]['Average Score'].append(stats[s]['avg'])
            data[j]['Max Score'].append(stats[s]['max'])
            data[j]['Min Score'].append(stats[s]['min'])
        data[n]['Iteration'].append(i+1)
        data[n]['Cumulative Score'].append(stats['players']['cumulative'])
        data[n]['Average Score'].append(stats['players']['avg'])
        data[n]['Max Score'].append(stats['players']['max'])
        data[n]['Min Score'].append(stats['players']['min'])
    dfs = [pd.DataFrame.from_dict(d) for d in data]
    for i in range(n):
        dfs[i].name = strategies[i]
    dfs[n].name = 'Agents'
    return dfs

def display_data(df):
    print(df.name)
    print(df)
    print(df.describe())

## Aléatoire uniforme et Tétue

In [12]:
dfs = df_interaction(2, ['uniformly_random','random_obstinate'])

In [19]:
display_data(dfs[0])

uniformly_random
   Average Score  Cumulative Score  Iteration  Max Score  Min Score
0            3.6                18          1          4          3
1            3.6                18          2          4          3
2            4.8                24          3          6          4
3            4.6                23          4          7          3
4            3.8                19          5          5          3
5            5.0                25          6          7          4
6            4.2                21          7          6          3
7            4.2                21          8          6          3
8            4.0                20          9          5          3
9            3.8                19         10          5          3
       Average Score  Cumulative Score  Iteration  Max Score  Min Score
count      10.000000         10.000000   10.00000  10.000000  10.000000
mean        4.160000         20.800000    5.50000   5.500000   3.200000
std         0.49710

In [20]:
display_data(dfs[1])

random_obstinate
   Average Score  Cumulative Score  Iteration  Max Score  Min Score
0           66.2               331          1         87         35
1           58.8               294          2         84         32
2           63.2               316          3         85         33
3           64.4               322          4         87         38
4           48.2               241          5         90         25
5           59.8               299          6         74         36
6           63.0               315          7         82         41
7           47.4               237          8         88         24
8           35.0               175          9         84         20
9           62.8               314         10         81         39
       Average Score  Cumulative Score  Iteration  Max Score  Min Score
count      10.000000         10.000000   10.00000  10.000000  10.000000
mean       56.880000        284.400000    5.50000  84.200000  32.300000
std        10.06653

In [21]:
display_data(dfs[2])

Agents
   Average Score  Cumulative Score  Iteration  Max Score  Min Score
0           34.9               349          1         87          3
1           31.2               312          2         84          3
2           34.0               340          3         85          4
3           34.5               345          4         87          3
4           26.0               260          5         90          3
5           32.4               324          6         74          4
6           33.6               336          7         82          3
7           25.8               258          8         88          3
8           19.5               195          9         84          3
9           33.3               333         10         81          3
       Average Score  Cumulative Score  Iteration  Max Score  Min Score
count      10.000000         10.000000   10.00000  10.000000  10.000000
mean       30.520000        305.200000    5.50000  84.200000   3.200000
std         5.082607         

# Pour aller plus loin

Idées exploratoires !

In [7]:
simulate(2,['min_occupation_forgetful','uniformly_random'])

{'players': {'cumulative': 44, 'avg': 4.4, 'max': 6, 'min': 3},
 'min_occupation_forgetful': {'cumulative': 23,
  'avg': 4.6,
  'max': 6,
  'min': 3},
 'uniformly_random': {'cumulative': 21, 'avg': 4.2, 'max': 5, 'min': 3},
 'restaurants': {'avg': 0.0},
 'restaurant 1': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'restaurant 2': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0